In [7]:
import json
with open("../data/raw/polymarket_20251109_2000.json") as f:
    data = json.load(f)

print(type(data))
print(data.keys())


<class 'dict'>
dict_keys(['data', 'next_cursor', 'limit', 'count'])


In [8]:
markets = data["data"]
print(len(markets))
print(markets[0].keys())
print(markets[0]["question"])
print(markets[0]["tokens"])


1000
dict_keys(['enable_order_book', 'active', 'closed', 'archived', 'accepting_orders', 'accepting_order_timestamp', 'minimum_order_size', 'minimum_tick_size', 'condition_id', 'question_id', 'question', 'description', 'market_slug', 'end_date_iso', 'game_start_time', 'seconds_delay', 'fpmm', 'maker_base_fee', 'taker_base_fee', 'notifications_enabled', 'neg_risk', 'neg_risk_market_id', 'neg_risk_request_id', 'icon', 'image', 'rewards', 'is_50_50_outcome', 'tokens', 'tags'])
NCAAB: Arizona State Sun Devils vs. Nevada Wolf Pack 2023-03-15
[{'token_id': '73470541315377973562501025254719659796416871135081220986683321361000395461644', 'outcome': 'Arizona State', 'price': 1, 'winner': True}, {'token_id': '56393761733830483601097051857899348522495376869600726893014309766300892311293', 'outcome': 'Nevada', 'price': 0, 'winner': False}]


In [10]:
m = markets[0]
print(m["question"])
print(m["tokens"])

NCAAB: Arizona State Sun Devils vs. Nevada Wolf Pack 2023-03-15
[{'token_id': '73470541315377973562501025254719659796416871135081220986683321361000395461644', 'outcome': 'Arizona State', 'price': 1, 'winner': True}, {'token_id': '56393761733830483601097051857899348522495376869600726893014309766300892311293', 'outcome': 'Nevada', 'price': 0, 'winner': False}]


In [19]:
import pandas as pd

records = []
for m in markets:
    q = m["question"]
    # iterate over the list of token dicts (not the literal string "tokens")
    for t in m.get("tokens", []):
        records.append({
            "question": q,
            "outcome": t.get("outcome"),
            "price": t.get("price"),
            "winner": t.get("winner", None),
            "closed": m.get("closed", None)
        })

df = pd.DataFrame(records)
print(df.head())


                                            question        outcome  price  \
0  NCAAB: Arizona State Sun Devils vs. Nevada Wol...  Arizona State    1.0   
1  NCAAB: Arizona State Sun Devils vs. Nevada Wol...         Nevada    0.0   
2  NCAAB: Tennessee Volunteers vs. Duke Blue Devi...      Tennessee    1.0   
3  NCAAB: Tennessee Volunteers vs. Duke Blue Devi...           Duke    0.0   
4      NBA: LA Clippers vs. Orlando Magic 2023-03-18       Clippers    0.0   

   winner  closed  
0    True    True  
1   False    True  
2    True    True  
3   False    True  
4   False    True  


In [21]:
df.info
df['closed'].value_counts(dropna=False)

closed
True     1882
False     118
Name: count, dtype: int64

In [53]:
df_active = df[df["closed"] == False]

print(df_active)


                                               question outcome  price  \
30    Will Newcastle beat Nottingham Forest? (03/17/...            0.0   
31    Will Newcastle beat Nottingham Forest? (03/17/...            0.0   
50    Will a mugshot of Donald Trump be released by ...            0.0   
51    Will a mugshot of Donald Trump be released by ...            0.0   
86    archWill Gavin Newsom win the 2024 US Presiden...     Yes    0.5   
...                                                 ...     ...    ...   
1921  Will Coinbase staking be available in the US t...            0.0   
1954  archWill Joe Biden win the 2024 US Presidentia...            0.0   
1955  archWill Joe Biden win the 2024 US Presidentia...            0.0   
1992  archWill Kamala Harris win the 2024 US Preside...            0.0   
1993  archWill Kamala Harris win the 2024 US Preside...            0.0   

      winner  closed  
30     False   False  
31     False   False  
50     False   False  
51     False   Fals

In [ ]:
df_active = df[(df["closed"] == False) | (df["closed"].isna())]

price_sums = (
    df_active.groupby("question")["price"]
    .sum()
    .reset_index(name="price_sum")
)

overpriced  = price_sums[price_sums["price_sum"] > 1]
underpriced = price_sums[price_sums["price_sum"] < 1]
print("Overpriced markets:")
print(overpriced.head())

print("\nUnderpriced markets:")
print(underpriced.head())

price_sums.sort_values("price_sum").head(10)  # most underpriced
price_sums.sort_values("price_sum", ascending=False).head(10)  # most overpriced


Overpriced markets:
Empty DataFrame
Columns: [question, price_sum]
Index: []

Underpriced markets:
                                            question  price_sum
0                       WILL $BUSD depeg by April 1?        0.0
1  Which party will control the U.S. Senate after...        0.0
2        Who will win Super Bowl LVII? (no overtime)        0.0
3  Will "Flowers" be #1 globally on Spotify for J...        0.0
4       Will "PredictIt Exchange" be live by June 1?        0.0
